In [2]:
import os
import glob
from dotenv import load_dotenv
import gradio as gr
from openai import OpenAI

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyCv


In [29]:
context = {}

employees = glob.glob('knowledge-base/employees/*')

for employee in employees:
    name = employee.split(' ')[-1][:-3]
    doc = ""
    with open(employee, "r") as f:
        doc = f.read()
    context[name] = doc

In [30]:
products = glob.glob('knowledge-base/products/*')

for product in products:
    name = product.split('/')[-1][:-3]
    doc = ""
    with open(product, "r") as f:
        doc = f.read()
    context[name] = doc

In [31]:
context.keys()

dict_keys(['Chen', 'Spencer', 'Tran', 'Blake', 'Lancaster', 'Thompson', 'Greene', 'Thomson', 'Trenton', 'Harper', 'Bishop', 'Carter', 'Rellm', 'Markellm', 'Homellm', 'Carllm'])

In [32]:
system_message = "You are an expert in answering accurate questions about Insurellm, the Insurance Tech company. Give brief, accurate answers. If you don't know the answer, say so. Do not make anything up if you haven't been provided with relevant context."

In [36]:
def get_relevant_context(message):
    relevant_context = []
    for context_title, context_details in context.items():
        if context_title in message:
            relevant_context.append(context_details)
    return relevant_context

In [37]:
print(get_relevant_context("Who is Avery Lancaster?"))

["# Avery Lancaster\n\n## Summary\n- **Date of Birth**: March 15, 1985  \n- **Job Title**: Co-Founder & Chief Executive Officer (CEO)  \n- **Location**: San Francisco, California  \n\n## Insurellm Career Progression\n- **2015 - Present**: Co-Founder & CEO  \n  Avery Lancaster co-founded Insurellm in 2015 and has since guided the company to its current position as a leading Insurance Tech provider. Avery is known for her innovative leadership strategies and risk management expertise that have catapulted the company into the mainstream insurance market.  \n\n- **2013 - 2015**: Senior Product Manager at Innovate Insurance Solutions  \n  Before launching Insurellm, Avery was a leading Senior Product Manager at Innovate Insurance Solutions, where she developed groundbreaking insurance products aimed at the tech sector.  \n\n- **2010 - 2013**: Business Analyst at Edge Analytics  \n  Prior to joining Innovate, Avery worked as a Business Analyst, focusing on market trends and consumer preferen

In [38]:
def add_context(message):
    relevant_context = get_relevant_context(message)
    if relevant_context:
        message += "\n\n Relevant information\n\n"
        for relevant in relevant_context:
            message += relevant + "\n\n"
    return message


In [39]:
print(add_context("Who is Alex Lancaster?"))

Who is Alex Lancaster?

 Relevant information

# Avery Lancaster

## Summary
- **Date of Birth**: March 15, 1985  
- **Job Title**: Co-Founder & Chief Executive Officer (CEO)  
- **Location**: San Francisco, California  

## Insurellm Career Progression
- **2015 - Present**: Co-Founder & CEO  
  Avery Lancaster co-founded Insurellm in 2015 and has since guided the company to its current position as a leading Insurance Tech provider. Avery is known for her innovative leadership strategies and risk management expertise that have catapulted the company into the mainstream insurance market.  

- **2013 - 2015**: Senior Product Manager at Innovate Insurance Solutions  
  Before launching Insurellm, Avery was a leading Senior Product Manager at Innovate Insurance Solutions, where she developed groundbreaking insurance products aimed at the tech sector.  

- **2010 - 2013**: Business Analyst at Edge Analytics  
  Prior to joining Innovate, Avery worked as a Business Analyst, focusing on marke

In [ ]:
def chat(message, history):
    messages = [{"role":"system", "content": system_message}]
    for user_message, assistant_message in history:
        messages.append({"role":"user", "content": user_message})
        messages.append({"role":"assistant", "content": assistant_message})

    message = add_context(message)
    messages.append({"role":"user", "content":message})

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chuck in stream:
        response += chuck.choices[0].delta.content or ''
        yield response

In [41]:
view = gr.ChatInterface(chat).launch()

/Users/sahachelflores/Desktop/LLM_Engineer/llm-engineering/llm/lib/python3.11/site-packages/gradio/chat_interface.py:338: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
